In [1]:
import os
import glob
import pandas as pd
import yfinance as yf
from datetime import timedelta, datetime

In [2]:
df = pd.DataFrame()
require_cols = ['Symbol', 'Trade Date','Trade Type','Quantity','Price','Order ID','Order Execution Time']

In [3]:
for f in glob.glob("tradesbook_exports/*.xlsx"):
    data = pd.read_excel(f, skiprows = 14, usecols = require_cols)
    df = df.append(data,ignore_index=True)
df.drop_duplicates('Order ID', inplace=True)
#df = df.drop(df[df['Trade Type'] == 'sell'].index)
df.reset_index(inplace=True)

C:\Users\abhij\AppData\Local\Temp\ipykernel_2900\1290121858.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data,ignore_index=True)


In [4]:
def find_indexPrice(date_,symbol):
    tradeDate=str(date_)
    ticker=symbol+'.NS'
    print(ticker)
    format = '%Y-%m-%d'
    start = datetime.strptime(tradeDate, format)
    end = start + timedelta(days=1)
    index_Price= yf.download('NIFTYBEES.NS',start,end)['Adj Close'].astype(int)
    try:
        LTP = yf.Ticker(ticker).history(period='1d')
        stock_LTP = LTP['Close'][0]
    except IndexError:
        try:
            LTP = yf.Ticker(ticker).history(period='3d')
            stock_LTP = LTP['Close'][0]
        except IndexError:
            stock_LTP = 0
    return index_Price[0],stock_LTP

In [5]:
index_ClosePrices,stock_LTP_Lst = [],[]
for numb,row in df.iterrows():
    index_Price,stock_LTP = find_indexPrice(row['Trade Date'],row['Symbol'])
    index_ClosePrices.append(index_Price)
    stock_LTP_Lst.append(stock_LTP)
df["index_ClosePrices"] = index_ClosePrices 
df["Stock_LTP"] = stock_LTP_Lst 
print(df)
df.to_excel("output.xlsx")

WIPRO.NS
[*********************100%***********************]  1 of 1 completed
SUBEX.NS
[*********************100%***********************]  1 of 1 completed
SUBEX.NS: No data found, symbol may be delisted
SUBEX.NS: No data found, symbol may be delisted
WIPRO.NS
[*********************100%***********************]  1 of 1 completed
CAMS.NS
[*********************100%***********************]  1 of 1 completed
CAMS.NS
[*********************100%***********************]  1 of 1 completed
SBIN.NS
[*********************100%***********************]  1 of 1 completed
LT.NS
[*********************100%***********************]  1 of 1 completed
HDFCBANK.NS
[*********************100%***********************]  1 of 1 completed
M&M.NS
[*********************100%***********************]  1 of 1 completed
SBIN.NS
[*********************100%***********************]  1 of 1 completed
LT.NS
[*********************100%***********************]  1 of 1 completed
SBIN.NS
[*********************100%*********************